## COLAB SETUP

In [1]:
# mount your drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
space = 'colab'

In [3]:
if space == 'colab':
    data_root = '/content/drive/MyDrive/Data'
    save_root = ''
else:
    data_root = 'C:/Users/james/Data/MIMIC/mimic-iii-clinical-database-1.4'
    save_root = 'C:/Users/james/Data/MIMIC/mimic-iii-chart-transformers'

In [4]:
!pip install x_transformers

  Created wheel for entmax: filename=entmax-1.0-cp37-none-any.whl size=11017 sha256=b8c75ada4934a489b972202acb81d2c54f3adbe182f2dad834db67fb8f622d73
  Stored in directory: /root/.cache/pip/wheels/4c/2c/4e/687c0abbeb16f906bd5fb8a9763e1cdd2b0d118ad55a4332f2
Successfully built entmax


## TENSORBOARD UTILS

In [5]:
%load_ext tensorboard

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

Do writing! e.g. see [PyTorch tutorial](https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html).

In [ ]:
tensorboard --logdir runs

## PREPROCESSING CODE

In [ ]:
import os
import numpy as np
import pandas as pd
import pickle as pickle
import torch
from sklearn.model_selection import train_test_split


In [ ]:
# paths

chartevents_path = os.path.join(data_root, "CHARTEVENTS.csv")
admissions_path = os.path.join(data_root,"ADMISSIONS.csv")
d_items_path = os.path.join(data_root, "d_items.csv")


In [ ]:
# read in admissions

admissions = pd.read_csv(admissions_path,
                         parse_dates=['ADMITTIME', 'DISCHTIME'])

# extract only those charted and apply labelling logic

charted = admissions[admissions.HAS_CHARTEVENTS_DATA == 1]
charted.drop('ROW_ID', axis=1, inplace=True)
charted['HADM_IN_SEQ'] = charted.groupby('SUBJECT_ID')['ADMITTIME'].rank().astype(int)
charted = charted.sort_values(by=['SUBJECT_ID', 'HADM_IN_SEQ'])
charted['ADMITTIME_NEXT'] = charted.groupby('SUBJECT_ID')['ADMITTIME'].shift(-1)
charted['DIS2ADM'] = charted['ADMITTIME_NEXT'] - charted['DISCHTIME']
charted['READM<7'] = (charted['DIS2ADM'] < pd.Timedelta(days=7)).astype(int)
charted['READM<30'] = (charted['DIS2ADM'] < pd.Timedelta(days=30)).astype(int)
charted.set_index('HADM_ID', inplace=True)

# get hadm_ids for the first admission

first_indices = charted[charted.HADM_IN_SEQ == 1].index.to_numpy()

# split first-hadm_ids into train, val, test and check.

train_indices, surplus = train_test_split(first_indices, train_size=0.8)
val_indices, test_indices = train_test_split(surplus, test_size=0.5)
del surplus
assert set(first_indices) == set(train_indices) | set(val_indices) | set(test_indices)

# helpers


def ts_to_posix(time):
    return pd.Timestamp(time, unit='s').timestamp()


def get_admittime(hadm_id):
    time = charted.loc[hadm_id, 'ADMITTIME']
    return ts_to_posix(time)


def get_from_charted(hadm_id, label):
    return charted.loc[hadm_id, label]


# token mappings

d_items = pd.read_csv(d_items_path)

token_shift = 1
pad_token = 0

itemid2token = dict(zip(d_items['ITEMID'], range(token_shift, token_shift + len(d_items))))

# add special tokens to the dictionary
itemid2token['[PAD]'] = pad_token
#itemid2token['[BOS]'] = 1
#itemid2token['[EOS]'] = 2
#itemid2token['[SEP]'] = 3

token2itemid = {v: k for k, v in itemid2token.items()}
token2label = dict(zip(range(len(d_items)), d_items['LABEL']))

with open(os.path.join(save_root, 'mappings.pkl'), 'wb') as f:
    pickle.dump({'itemid2token': itemid2token,
                 'token2itemid': token2itemid},
                f)


def map2token(itemid):
    return itemid2token[np.int(itemid)]


def map2itemid(token):
    return str(token2itemid[token])


def map2itemidstr(tokens):
    return ' '.join(list(map(map2itemid, tokens)))


# loop through sets and generate output files

for subset in ['val', 'train', 'test']:
    print(f'Processing {subset} set data...')

    # grouper for charts

    gpdf = (pd.read_csv(chartevents_path, skiprows=0, 
                        nrows=10000000 if space != 'colab',
                        header=0,
                        usecols=['HADM_ID', 'CHARTTIME', 'ITEMID'],
                        dtype={'HADM_ID': np.int},
                        converters={'ITEMID': map2token},
                        parse_dates=['CHARTTIME'])
            .query(f'HADM_ID.isin(@{subset}_indices)')
            .groupby(by='HADM_ID')
            )

    # initialise

    tokens = dict()
    times = dict()
    times_rel = dict()
    labels = dict()

    # populate with entries

    for i in gpdf.groups:
        time_origin = get_admittime(i)
        temp = gpdf.get_group(i).sort_values(by="CHARTTIME")
        tokens[i] = np.array(temp['ITEMID'], dtype=int)
        times[i] = np.fromiter(
            map(ts_to_posix, temp['CHARTTIME']),
            dtype=np.int64
        )
        times_rel[i] = times[i] - time_origin
        labels[i] = {
            'readm_7': get_from_charted(i, 'READM<7'),
            'readm_30': get_from_charted(i, 'READM<30')
        }

    # write out charts to pickle

    save_path = os.path.join(save_root, f'{subset}_charts.pkl')

    with open(save_path, 'wb') as f:
        pickle.dump({f'{subset}_tokens': tokens,
                     f'{subset}_times': times,
                     f'{subset}_times_rel': times_rel}, f)

    del tokens, times, times_rel, gpdf

    # write out labels to pickle

    save_path = os.path.join(save_root, f'{subset}_labels.pkl')

    with open(save_path, 'wb') as f:
        pickle.dump({f'{subset}_labels': labels}, f)

    del labels

    print(f'{subset} set data processed!')



SyntaxError: ignored

## SELF-SUPERVISED MODE



### THE MODEL

In [7]:
import os
import copy
import tqdm
import random

import numpy as np
import pandas as pd
import pickle as pickle
import torch

from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.nn as nn
from x_transformers import TransformerWrapper, Decoder
from x_transformers.autoregressive_wrapper import AutoregressiveWrapper


In [8]:
# paths

train_path = os.path.join(data_root, "train_charts.pkl")
val_path = os.path.join(data_root, "val_charts.pkl")
mapping_path = os.path.join(data_root, "mappings.pkl")

# misc

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# token mappings:  # TODO: refactor to module where possible.

with open(mapping_path, 'rb') as f:
    mappings = pickle.load(f)
    itemid2token = mappings['itemid2token']
    token2itemid = mappings['token2itemid']
    del mappings

num_tokens = len(itemid2token)

# token mappings: decoders


def decode_token(token):
    return str(token2itemid[token])


def decode_tokens(tokens):
    return ' '.join(list(map(decode_token, tokens)))


# get data

def fetch_data(path, var_key):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data[var_key]


trX = fetch_data(train_path, 'train_tokens')
vaX = fetch_data(val_path, 'val_tokens')

data_train = {k: torch.from_numpy(v) for k, v in trX.items()}
data_val = {k: torch.from_numpy(v) for k, v in vaX.items()}


# yield from loader

def cycle(loader):
    while True:
        for data in loader:
            yield data


# constants  # TODO: consider having these stored in checkpoint

NUM_EPOCHS = 5
NUM_BATCHES = 1000 #int(1e5)
BATCH_SIZE = 4
GRADIENT_ACCUMULATE_EVERY = 4  # 4
LEARNING_RATE = 1e-4
VALIDATE_EVERY = 10
CHECKPOINT_AFTER = 10
GENERATE_EVERY = 100
GENERATE_LENGTH = 200
SEQ_LEN = 200

# instantiate GPT-like decoder model

model = TransformerWrapper(
    num_tokens=num_tokens,  # Expects each val in data to be [0, num_tokens)
    max_seq_len=SEQ_LEN,
    attn_layers=Decoder(dim=300, depth=3, heads=8)  # full: 512, 6, 8
)

pre_model = AutoregressiveWrapper(model)
pre_model.to(device)


# custom sequence-excerpt sampler

class SeqSamplerDataset(Dataset):  # TODO: tidy __getitem__ method
    def __init__(self, data, seq_len):
        super().__init__()  # gives access to Dataset methods.
        self.data = data
        self.seq_len = seq_len
        self.lookup = dict(zip(np.arange(len(self.data)),
                               self.data.keys()))

    def __getitem__(self, key):  # a.t.m. when data[key] shorter length than SEQ_LEN, padded with 0.
        item_len = self.data[self.lookup[key]].size(0)
        rand_start = torch.randint(0, item_len - self.seq_len, (1,)) if item_len > self.seq_len else 0
        lenfromseq = min(item_len, self.seq_len)
        sample = torch.zeros(self.seq_len)
        sample[:lenfromseq] = self.data[self.lookup[key]][rand_start: rand_start + lenfromseq]
        sample = sample.long()
        return sample.to(device)

    def __len__(self):
        return len(self.data)


class ClsSamplerDataset(Dataset):  # TODO: tidy __getitem__ method with more natural pad operations.
    def __init__(self, data, seq_len, labels=None):
        super().__init__()
        self.data = data
        self.labels = labels
        self.seq_len = seq_len
        self.lookup = dict(zip(np.arange(len(self.data)), self.data.keys()))

    def __getitem__(self, key):  # a.t.m. when data[key] shorter length than SEQ_LEN, padded with 0.
        index = self.lookup[key]
        item_len = self.data[index].size(0)
        rand_start = torch.randint(0, item_len - self.seq_len, (1,)) if item_len > self.seq_len else 0
        lenfromseq = min(item_len, self.seq_len)
        sample = torch.zeros(self.seq_len)
        sample[:lenfromseq] = self.data[index][rand_start: rand_start + lenfromseq]

        if self.labels is not None:
            label = torch.tensor(self.labels[index])
            return sample.long().to(device), label.long().to(device)
        else:
            return sample.long().to(device)

    def __len__(self):
        return len(self.data)


train_dataset = ClsSamplerDataset(data_train, SEQ_LEN)
val_dataset   = ClsSamplerDataset(data_val, SEQ_LEN)

train_loader  = cycle(DataLoader(train_dataset, batch_size=BATCH_SIZE))
val_loader    = cycle(DataLoader(val_dataset,   batch_size=BATCH_SIZE))


### TRAINING LOOP

In [12]:
optim = torch.optim.Adam(pre_model.parameters(), lr=LEARNING_RATE)
ckpt_path = os.path.join(save_root, "pre_model_0.pt")

writer = SummaryWriter(log_dir=f"runs/mimic_exp1/pre")

# training loop

best_val_loss = np.inf

for epoch in range(NUM_EPOCHS):
  for i in tqdm.tqdm(range(NUM_BATCHES), mininterval=10., desc='training'):
      pre_model.train()

      for __ in range(GRADIENT_ACCUMULATE_EVERY):
          loss = pre_model(next(train_loader))
          loss.backward()

      torch.nn.utils.clip_grad_norm_(pre_model.parameters(), 0.5)
      optim.step()
      optim.zero_grad()

      # parameter tracking

      writer.add_scalar('loss',
                        loss.item(),
                        epoch * NUM_BATCHES + i
                        )

      # validate model

      if i % VALIDATE_EVERY == 0:
          pre_model.eval()
          with torch.no_grad():
              val_loss = pre_model(next(val_loader)).item()
              if val_loss < best_val_loss:
                  print(f'VL: {val_loss} < BVL: {best_val_loss}')
                  best_val_loss = val_loss
              

      # checkpoint model

      if ((i > CHECKPOINT_AFTER) & (val_loss < best_val_loss)):
          print("Checkpoint saving...")
          torch.save({
              'train_step': i,
              'model_state_dict': pre_model.state_dict(),
              'SEQ_LEN': SEQ_LEN,
              'optim_state_dict': optim.state_dict(),
              'val_loss': val_loss
          }, ckpt_path)
          print("Checkpoint saved!\n")
              

      # generate sequence

      if i % GENERATE_EVERY == 0:
          pre_model.eval()
          inp = random.choice(val_dataset)[:-1]
          primer_str = decode_tokens(inp.cpu().numpy())
          print('primer:', primer_str, '*' * 100, sep='\n')

          sample = pre_model.generate(inp, GENERATE_LENGTH)
          sample_str = decode_tokens(sample.cpu().numpy())
          print('output:', sample_str, '\n', sep='\n')

training:   0%|          | 0/1000 [00:00<?, ?it/s]

VL: 8.019222259521484 < BVL: inf
primer:
220277 220210 220181 220180 220179 220050 220052 220051 220277 220045 220074 223761 225664 224366 220180 220179 220074 220052 220051 220050 220045 220210 220277 220181 220277 220210 220181 220179 220180 220052 220051 220050 220045 220074 224366 223761 220277 220210 220181 220180 220052 220074 220051 220050 220045 220179 220277 224366 220210 220074 220051 220050 220045 220052 220045 220050 220051 220052 220074 220210 220277 220074 225664 223761 220277 220210 220045 220051 220050 220052 220210 224366 220277 220074 220051 220050 220045 220052 220277 220210 220074 220045 220051 220050 220052 220045 220050 220051 220052 220074 220210 220277 220210 220277 220074 224366 220051 220050 220045 220052 223761 220277 220210 220074 220051 220050 220045 220052 224366 220277 220210 220074 220052 220051 220050 220045 225625 220621 220635 220645 225624 225677 227073 227442 227443 227457 220602 220546 220615 220228 220545 220277 220210 220074 220045 220052 220051 

training:   6%|▌         | 57/1000 [00:10<02:48,  5.61it/s]

VL: 3.588179349899292 < BVL: 5.109420299530029
VL: 3.4264373779296875 < BVL: 3.588179349899292
VL: 2.543121099472046 < BVL: 3.4264373779296875
primer:
225668 225668 220545 220615 220621 220632 220635 220645 225624 225625 225677 227073 227442 227443 220602 226063 226062 225668 223679 220274 227471 220734 225668 227686 225668 227457 227443 227442 227073 225677 225625 220545 220645 220635 220621 220546 220602 220228 225624 220615 225668 220546 220545 227457 227443 227442 225677 225625 227073 220645 220635 220621 220615 220602 225624 220228 225668 220602 220228 220545 220546 220615 220645 225624 227073 227442 227443 227457 220621 227443 227442 227073 225624 220645 220602 220615 220546 220545 220228 220621 227457 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

training:  11%|█▏        | 114/1000 [00:20<02:37,  5.63it/s]

...

training:  79%|███████▉  | 793/1000 [02:14<00:34,  5.98it/s]

primer:
220545 227467 227466 227443 227442 227073 225677 227465 225624 220645 220635 220621 220615 220602 225625 220545 227442 227443 227454 227467 227465 227466 227073 227457 225677 220546 225624 220645 220635 220621 220615 220602 220545 220228 225625 227471 220734 220545 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 

training:  86%|████████▌ | 855/1000 [02:24<00:24,  5.91it/s]

primer:
220545 227467 227466 227465 227457 227443 227442 227073 225677 225625 220645 220635 220621 220615 220602 220546 220545 220228 225624 220734 227471 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]

training: 100%|██████████| 1000/1000 [02:48<00:00,  5.92it/s]


### GENERATING SEQUENCES

In [ ]:
# reading d_items for interpretability 
d_items_path = os.path.join(data_root, "d_items.csv")
d_items = pd.read_csv(d_items_path, index_col='ITEMID', dtype={'ITEMID': str})

In [ ]:
def decode_token(token):
    return str(token2itemid[token])

def decode_tokens(tokens):
    return ' '.join(list(map(decode_token, tokens)))

def token2label(token):
    itemid = token2itemid[token]
    try:
        x = d_items.loc[token, 'LABEL']
    except KeyError:
        x = '......'
    return x

def tokens2labels(tokens):
    return ' -> '.join(list(map(token2label, tokens)))

In [ ]:
# fetch model weights
weights_path = os.path.join(data_root, 'pre_model.pt')
X = torch.load(weights_path, map_location=device)
states = X['model_state_dict']
base_states = { k[len('net.'):] if k[:len('net.')] == 'net.' else k : v for k, v in states.items()}

In [ ]:
pre_model.load_state_dict(states)
pre_model.to(device)

pre_model.eval()
with torch.no_grad():
    prompt = random.choice(val_dataset)[0:5]
    #prompt = torch.tensor([2, 2, 2]).to(device)
    sample = pre_model.generate(start_tokens=prompt, seq_len = 20, eos_token = 0)
    print(tokens2labels(prompt.cpu().numpy()[::-1]))
    print(decode_tokens(prompt.cpu().numpy()[::-1]), '\n')

    print(tokens2labels(sample.cpu().numpy()[::-1]), '\n')
    print(decode_tokens(sample.cpu().numpy()[::-1]))


...... -> Bilateral hernia dsg -> Bed alarm -> total RR -> ......
224162 220181 220180 220179 220045 

...... -> Fentanyl: -> Bed alarm -> Bilateral hernia dsg -> total RR -> ...... -> inital bath -> inital bath -> INV#3 Dsg Change [MM] -> Bilateral hernia dsg -> Bed alarm -> total RR -> ...... -> inital bath -> INV#3 Dsg Change [MM] -> ...... -> ...... -> ...... -> KCl Supps -> Fentanyl: 

223770 224161 220180 220181 220179 220045 220277 220277 220210 220181 220180 220179 220045 220277 220210 223761 223752 223751 226253 224161


## FINE-TUNING MODE

In [ ]:
class FinetuningWrapper(nn.Module):
    def __init__(self, net, num_classes, state_dict = None,
                 ignore_index = -100, pad_value = 0):
        super().__init__()
        self.pad_value = pad_value
        self.ignore_index = ignore_index
        self.num_classes = num_classes

        self.net = copy.deepcopy(net)
        self.max_seq_len = self.net.max_seq_len

        # initialise net from pretrained
        if state_dict is not None:
            self.net.load_state_dict(state_dict)

        # define classifier head layers
        self.num_features = net.to_logits.in_features * 200
        self.net.clf1 = nn.Linear(self.num_features, num_classes, bias=True)

    def forward(self, X, Y, **kwargs):
        Z = self.net(X, return_embeddings=True, **kwargs)
        Z = torch.flatten(Z, start_dim=1)
        logits = self.net.clf1(Z)
        loss = F.cross_entropy(logits, Y)
        return loss

In [ ]:
train_lbl_path = os.path.join(save_root, "train_labels.pkl")
val_lbl_path = os.path.join(save_root, "val_labels.pkl")
FT_BATCH_SIZE = 100

# fetch labels
with open(train_lbl_path, 'rb') as f:
    X = pickle.load(f)
    train_labels = {k: v['readm_30'] for k, v in  X['train_labels'].items()}
    del X

with open(val_lbl_path, 'rb') as f:
    X = pickle.load(f)
    val_labels = {k: v['readm_30'] for k, v in  X['val_labels'].items()}
    del X

# generate datasets and loaders
ft_train_dataset = ClsSamplerDataset(data_train, SEQ_LEN, labels=train_labels)
ft_val_dataset = ClsSamplerDataset(data_val, SEQ_LEN, labels=val_labels)

ft_train_loader = cycle(DataLoader(ft_train_dataset, batch_size=FT_BATCH_SIZE))
ft_val_loader = cycle(DataLoader(ft_val_dataset, batch_size=FT_BATCH_SIZE))

In [ ]:
# fetch model weights
weights_path = os.path.join(save_root, 'pre_model.pt')
X = torch.load(weights_path)
states = X['model_state_dict']
base_states = { k[len('net.'):] if k[:len('net.')] == 'net.' else k : v for k, v in states.items()}

### TRAINING LOOP

In [ ]:
# initialisation

model_ft = FinetuningWrapper(model, num_classes=2, state_dict=base_states)
model_ft.to(device)

optim_ft = torch.optim.Adam(model_ft.parameters(), lr=0.001)
ckpt_ft_path = os.path.join(save_root, "model_ft2.pt")

NUM_FT_BATCHES = 100

# training loop

best_val_loss = np.inf
for i in tqdm.tqdm(range(NUM_FT_BATCHES), mininterval=10., desc='fine-tuning'):
    model_ft.train()

    for __ in range(GRADIENT_ACCUMULATE_EVERY):
        X, Y = next(ft_train_loader)
        loss = model_ft(X, Y)
        loss.backward()

    print(f'training loss: {loss.item()}')
    torch.nn.utils.clip_grad_norm_(model_ft.parameters(), 0.5)
    optim_ft.step()
    optim_ft.zero_grad()

    # validate model_ft

    if i % VALIDATE_EVERY == 0:
        model_ft.eval()
        with torch.no_grad():
            X, Y = next(ft_val_loader)
            val_loss = model_ft(X, Y)
            print(f'validation loss: {val_loss.item()}')
            if val_loss.item() < best_val_loss:
                best_val_loss = val_loss.item()

        if (i > CHECKPOINT_AFTER) & (val_loss.item() < best_val_loss):
                torch.save({
                    'train_step': i,
                    'model_state_dict': model_ft.state_dict(),
                    'SEQ_LEN': SEQ_LEN,
                    'optim_state_dict': optim_ft.state_dict(),
                    'val_loss': val_loss
                }, ckpt_ft_path)
                print("Checkpoint saved!\n")

    # generate sequence

    #if i % GENERATE_EVERY == 0:
    #    model.eval()
    #    inp = random.choice(val_dataset)[:-1]
    #    primer_str = decode_tokens(inp.numpy())
    #    print('primer:', primer_str, '*' * 100, sep='\n')
    #
    #    sample = model.generate(inp, GENERATE_LENGTH)
    #    sample_str = decode_tokens(sample.numpy())
    #    print('output:', sample_str, sep='\n')





fine-tuning:   0%|          | 0/1000 [00:00<?, ?it/s]

training loss: 0.7750842571258545
validation loss: 2.330962896347046
training loss: 1.7908108234405518
training loss: 17.393821716308594
training loss: 0.6341850757598877
training loss: 0.4809987545013428
training loss: 0.6009710431098938
training loss: 0.40837374329566956
training loss: 1.264967441558838
training loss: 3.9137308597564697
training loss: 0.48501887917518616
training loss: 0.08094941824674606
validation loss: 0.5570998787879944
training loss: 0.42756617069244385





fine-tuning:   1%|▏         | 13/1000 [00:10<12:58,  1.27it/s]

training loss: 0.4682634770870209
training loss: 0.08903224021196365
training loss: 0.43009358644485474
training loss: 0.2945011854171753
training loss: 0.41261324286460876
training loss: 0.2123783826828003
training loss: 0.23356439173221588
training loss: 0.3060793876647949
training loss: 0.32110539078712463
validation loss: 0.2914092540740967
training loss: 0.26054316759109497
training loss: 0.19257989525794983
training loss: 0.3618169128894806
training loss: 0.15792053937911987
training loss: 0.24263876676559448





fine-tuning:   3%|▎         | 27/1000 [00:20<12:40,  1.28it/s]

...




fine-tuning:   4%|▍         | 41/1000 [00:31<12:27,  1.28it/s]

training loss: 0.36697161197662354
validation loss: 0.17502136528491974
training loss: 0.3087913691997528
training loss: 0.17607274651527405
training loss: 0.31863605976104736
training loss: 0.15240712463855743
training loss: 0.13296151161193848
training loss: 0.11476421356201172
training loss: 0.11663253605365753
training loss: 0.13183915615081787
training loss: 0.47518742084503174
training loss: 0.07362198084592819
validation loss: 0.40637633204460144
training loss: 0.15867266058921814
training loss: 0.23520687222480774





fine-tuning:   5%|▌         | 54/1000 [00:41<12:14,  1.29it/s]

training loss: 0.15543578565120697
training loss: 0.25696900486946106
training loss: 0.3110707402229309
training loss: 0.3178585171699524
training loss: 0.21875987946987152
training loss: 0.14336532354354858
training loss: 0.2208518236875534
training loss: 0.30784863233566284
validation loss: 0.23428674042224884
training loss: 0.29378435015678406
training loss: 0.20915424823760986
training loss: 0.3731994330883026
training loss: 0.21939434111118317
training loss: 0.10782883316278458
training loss: 0.15824007987976074


KeyboardInterrupt: ignored